In [4]:
import spacy
import nltk
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest

In [5]:
# Завантаження необхідних ресурсів NLTK
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

True

In [8]:
def generate_summary(text, select_length_ratio=0.3):
    """
    Генерує резюме тексту на основі частоти вживання слів.
    """
    # 1. Завантаження мовної моделі SpaCy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)

    # 2. Підготовка стоп-слів та пунктуації
    stop_words = set(stopwords.words('english'))
    punctuation_list = list(punctuation) + ['\n']

    # 3. Підрахунок частоти появи слів
    word_frequencies = {}
    for word in doc:
        word_text = word.text.lower()
        if word_text not in stop_words and word_text not in punctuation_list:
            if word_text not in word_frequencies.keys():
                word_frequencies[word_text] = 1
            else:
                word_frequencies[word_text] += 1

    # Нормалізація частоти (ділимо на максимальну частоту для отримання ваги від 0 до 1)
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    # 4. Токенізація на речення та розрахунок їхньої ваги
    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}
    
    for sent in sentence_tokens:
        for word in sent:
            word_text = word.text.lower()
            if word_text in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word_text]
                else:
                    sentence_scores[sent] += word_frequencies[word_text]

    # 5. Вибір найважливіших речень за допомогою heapq
    # Розраховуємо кількість речень для резюме (наприклад, 30% від оригіналу)
    select_length = max(1, int(len(sentence_tokens) * select_length_ratio))
    
    summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    
    # Формування фінального тексту
    final_summary = [sent.text for sent in summary_sentences]
    return " ".join(final_summary)

In [10]:
# Вхідний текст про шаттл Discovery
text = """The Orbiter Discovery, OV-103, is considered eligible for listing in the National 
Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program 
(1969-2011) under Criterion A in the areas of Space Exploration and Transportation 
and under Criterion C in the area of Engineering. Because it has achieved significance 
within the past fifty years, Criteria Consideration G applies. Under Criterion A, 
Discovery is significant as the oldest of the three extant orbiter vehicles constructed 
for the Space Shuttle Program (SSP), the longest running American space program to date; 
she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo 
programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually 
the construction of a space station. Including her maiden voyage (launched August 30, 1984), 
Discovery flew to space thirty-nine times, more than any of the other four orbiters; 
she was also the first orbiter to fly twenty missions. She had the honor of being chosen 
as the Return to Flight vehicle after both the Challenger and Columbia accidents. 
Discovery was the first shuttle to fly with the redesigned SRBs, a result of the 
Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. 
Discovery also carried the Hubble Space Telescope to orbit and performed two of the 
five servicing missions to the observatory. She flew the first and last dedicated 
Department of Defense (DoD) missions, as well as the first unclassified defense-related 
mission. In addition, Discovery was vital to the construction of the International 
Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to 
the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, 
and the first to perform an exchange of a resident crew. Under Criterion C, Discovery 
is significant as a feat of engineering. According to Wayne Hale, a flight director 
from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological 
leap from expendable rockets and capsules to a reusable, winged, hypersonic, 
cargo-carrying spacecraft.” Although her base structure followed a conventional 
aircraft design, she used advanced materials that both minimized her weight for 
cargo-carrying purposes and featured low thermal expansion ratios, which provided 
a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle 
orbiter also featured the first reusable TPS; all previous spaceflight vehicles had 
a single-use, ablative heat shield. Other notable engineering achievements of the 
orbiter included the first reusable orbital propulsion system, and the first 
two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle 
remains “the largest, fastest, winged hypersonic aircraft in history,” having 
regularly flown at twenty-five times the speed of sound."""

In [13]:
# Виконання
result = generate_summary(text)
print("--- ТЕКСТОВЕ РЕЗЮМЕ ---")
print(result)

--- ТЕКСТОВЕ РЕЗЮМЕ ---
The Orbiter Discovery, OV-103, is considered eligible for listing in the National 
Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program 
(1969-2011) under Criterion A in the areas of Space Exploration and Transportation 
and under Criterion C in the area of Engineering. Under Criterion A, 
Discovery is significant as the oldest of the three extant orbiter vehicles constructed 
for the Space Shuttle Program (SSP), the longest running American space program to date; 
she was the third of five orbiters built by NASA. According to Wayne Hale, a flight director 
from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological 
leap from expendable rockets and capsules to a reusable, winged, hypersonic, 
cargo-carrying spacecraft.” In addition, Discovery was vital to the construction of the International 
Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to 
the station by a U.S. Space S